In [132]:
import anyjson as json
import requests
from bs4 import BeautifulSoup
#import scalpyr
import urllib
from json import load 
import pandas as pd
from pandas import DataFrame
import numpy as np
import codecs
#from urllib import request
from urllib2 import urlopen # python 2.7 version
#import urlopen
from json import load
#import sqlite3
#import csv
#import codecs
#import cStringIO
#import sys
import hashlib

In [102]:
api_key = 'MDE5OTYyMjIxMDE0Mzc2OTI5ODJhMTY0Ng001'
output = 'JSON'

In [103]:
import re
def cleanName(name):
    """
    Use regular expressions to change an artist name string into
    a valid seatgeek slug.
    """
    s = name
    if ', The' in s:
        s = 'The ' + s[:-5]
    #r = re.compile(r"\s+")
    #s = r.sub("-", s) # "a\nb\nc
    return s.lower()

In [152]:
# get list of artists from npr
url = 'http://api.npr.org/list?id=3009&output=JSON&apiKey=MDE5OTYyMjIxMDE0Mzc2OTI5ODJhMTY0Ng001'
reader = codecs.getreader("utf-8")

response = urlopen(url)
json_obj = load(response)

json_obj.keys()
#json_obj[u'subcategory'][27]['item'][1]

# Create a dictionary indexed by artist name
alist = []
for i in range(len(json_obj[u'subcategory'])):
    alist += json_obj[u'subcategory'][i]['item']
npr_artist_ids = {a['title']['$text']:a['id'] for a in alist}

npr_artist_ids['The Black Keys']

u'15193815'

In [143]:
# mappings copied from http://www.npr.org/api/mappingCodes.php
mapping = pd.DataFrame.from_csv('mapping.tsv',sep='\t',index_col=None)
mapping

,artist_name,id
0,Lee Rocker,14873692
1,Feisty,114900320
2,Michelle Shocked,14949553
3,Xavier Rudd,14950013
4,Matisyahu,14954504
5,Tracy Grammer,14954870
6,Shannon McNally,14983148
7,West Indian Girl,14983401
8,Chicago Blues Reunion,14990423
9,Orchestra of St. Luke's,14996042


In [113]:
ArtistList = pd.DataFrame.from_csv('../billboard/billboard_artist_names_2010_15.csv',index_col=None)

ArtistList['name_length'] = 0
for i in range(len(ArtistList)):
    ArtistList['name_length'][i] = len(ArtistList['artist_name'][i])
ArtistList = ArtistList[ArtistList['name_length'] <= 60]
ArtistList['artist_name_clean'] = ArtistList.artist_name.apply(cleanName)
ArtistList.head()

/Users/swahl/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,artist_name,artist_name_clean,artist_name_underscore,artist_name_query,artist_name_dash,name_length
0,Ke$ha,ke$ha,ke$ha,ke$ha,ke$ha,5
1,Eminem,eminem,eminem,eminem,eminem,6
2,Katy Perry,katy perry,katy_perry,katy+perry,katy-perry,10
3,Rihanna,rihanna,rihanna,rihanna,rihanna,7
4,Bruno Mars,bruno mars,bruno_mars,bruno+mars,bruno-mars,10


In [155]:
# create table of stories
#artists = ['The Black Keys','Radiohead','Lady Gaga']
#artists = ArtistList.iloc[:10].artist_name_clean
artists = ArtistList.artist_name_clean

npr_shows = pd.DataFrame()

for a in artists:
    
    try:
        #artist_id = npr_artist_ids[a]
        artist_id = str(int(mapping[mapping.artist_name == a].id))
        print artist_id
    except:
        print 'No id for:',a
        
    else:
        url = 'http://api.npr.org/query?id='+artist_id+'&output='+output+'&apiKey='+api_key
        print url
        reader = codecs.getreader("utf-8")

        try:
            response = urlopen(url)
            json_obj = load(response)

            stories = json_obj['list']['story']
        except:
            print 'Problem encountered with query.'
        else:

            cols = ['show_id','artist','artist_id','title','show',
                    'storyDate','pubDate','lastModifiedDate','link']#,'keywords']
            df = pd.DataFrame()

            for story in stories:
                try:
                    df = df.append(
                         pd.DataFrame([[
                             story['id'],
                             a,
                             artist_id,
                             story['title'][u'$text'],
                             story['slug'][u'$text'],
                             story['storyDate'][u'$text' ],
                             story['pubDate'][u'$text' ],
                             story['lastModifiedDate'][u'$text'],
                             story['link'][0][u'$text']#,story['keywords']
                                ]],columns=cols ) 
                        )
                except:
                    pass
                
            npr_shows = npr_shows.append(df)

# convert datetimes
for f in ['storyDate','pubDate','lastModifiedDate']:
    npr_shows[f] = pd.to_datetime(npr_shows[f])
npr_shows.head()

129141499
http://api.npr.org/query?id=129141499&output=JSON&apiKey=MDE5OTYyMjIxMDE0Mzc2OTI5ODJhMTY0Ng001
15404635
http://api.npr.org/query?id=15404635&output=JSON&apiKey=MDE5OTYyMjIxMDE0Mzc2OTI5ODJhMTY0Ng001
91837552
http://api.npr.org/query?id=91837552&output=JSON&apiKey=MDE5OTYyMjIxMDE0Mzc2OTI5ODJhMTY0Ng001
15757248
http://api.npr.org/query?id=15757248&output=JSON&apiKey=MDE5OTYyMjIxMDE0Mzc2OTI5ODJhMTY0Ng001
166969902
http://api.npr.org/query?id=166969902&output=JSON&apiKey=MDE5OTYyMjIxMDE0Mzc2OTI5ODJhMTY0Ng001
15817733
http://api.npr.org/query?id=15817733&output=JSON&apiKey=MDE5OTYyMjIxMDE0Mzc2OTI5ODJhMTY0Ng001
Problem encountered with query.
No id for: Far East Movement
No id for: B.o.B 
No id for: The Black Eyed Peas
No id for: Taio Cruz
96951070
http://api.npr.org/query?id=96951070&output=JSON&apiKey=MDE5OTYyMjIxMDE0Mzc2OTI5ODJhMTY0Ng001
No id for: Lady Antebellum
No id for: Iyaz
No id for: Young Money
No id for: Artists For Haiti
120581188
http://api.npr.org/query?id=120581188&o

,show_id,artist,artist_id,title,show,storyDate,pubDate,lastModifiedDate,link
0,137041753,Ke$ha,129141499,"For 2011, The Summer Concert Season Gets A Reboot",The Record,2011-06-08 19:45:00,2011-06-08 22:12:00,2011-06-10 16:58:09,http://www.npr.org/sections/therecord/2011/06/...
0,131968651,Ke$ha,129141499,"The 100 Biggest Songs Of 2010: Drinking, Danci...",The Record,2010-12-13 15:00:00,2010-12-13 23:34:00,2010-12-13 23:34:42,http://www.npr.org/sections/therecord/2010/12/...
0,129992395,Ke$ha,129141499,The Secret Of Dr. Luke's Success (Might Stay A...,The Record,2010-09-20 17:00:00,2010-09-20 18:00:00,2010-09-20 18:00:28,http://www.npr.org/sections/therecord/2010/09/...
0,129134759,Ke$ha,129141499,Pop Personae: Why Do Some Women Perform In Cha...,The Record,2010-08-12 04:00:00,2012-05-23 17:11:00,2012-06-06 14:45:46,http://www.npr.org/sections/therecord/2010/08/...
0,114630495,Ke$ha,129141499,Surprised That The 'Tik-Tok' Guy Won An ASCAP ...,Monkey See,2010-04-24 15:47:00,2010-08-12 00:24:00,2010-08-12 00:24:28,http://www.npr.org/sections/monkeysee/2010/04/...


In [157]:
npr_shows.to_csv('npr_shows_from_billboard_2010_15.csv',index=False,encoding="utf-8")
